FUNCIÓN LAMBDA PARA LECTURA DIARIA DE LAS PREDICCIONES
Este código se pone en AWS_Lambda para 


In [ ]:
"BUCKET_NAME"=BUCKET_NAME
import boto3
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

def lambda_handler(event, context):
    # Nombre del archivo de tu Jupyter Notebook en S3
    nombre_archivo_s3 = "BUCKET_NAME"
    # Nombre del archivo de salida después de la conversión
    nombre_archivo_salida = "output.py"
    
    # Configurar el cliente de S3
    s3 = boto3.client('s3')
    
    # Descargar el archivo del Jupyter Notebook desde S3
    with open('/tmp/notebook.ipynb', 'wb') as data:
        s3.download_fileobj('BUCKET_NAME', nombre_archivo_s3, data)
        
    
    # Leer el contenido del archivo del Jupyter Notebook
    with open('/tmp/notebook.ipynb', 'r') as f:
        nb = nbformat.read(f, as_version=4)
    
    # Crear un ejecutor preprocesador
    preprocessor = ExecutePreprocessor(timeout=600, kernel_name='python3')
    
    # Ejecutar el archivo del Jupyter Notebook
    try:
        preprocessor.preprocess(nb, {'metadata': {'path': '/tmp/'}})
    except Exception as e:
        print(e)
    
    # Guardar el resultado de la ejecución en un archivo Python
    with open(f'/tmp/{nombre_archivo_salida}', 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)
    
    # Subir el archivo resultante a S3
    with open(f'/tmp/{nombre_archivo_salida}', 'rb') as data:
        s3.upload_fileobj(data, 'tfmalicante', nombre_archivo_salida)
    
    return {
        'statusCode': 200,
        'body': 'Proceso completado exitosamente.'
    }